# テキスト生成のためにゼロショットプロンプトを使用して Bedrock モデルを呼び出す

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*


## はじめに

このノートブックでは、カスタマーサポートエンジニアから受けたカスタマーサービスの品質について、否定的なフィードバックを提供した顧客への返信メールを生成するために、LLMを使用する方法を示します。

ここでは、Boto3 APIを使用して、Bedrock の Amazon Titan Text Express モデルを使用します。

この例で使用されているプロンプトは、プロンプト以外の例文などを提供していないため、ゼロショットプロンプトと呼ばれます。

**注:** *このノートブックは、AWS環境の内外で実行できます。*

#### コンテキスト
Amazon Bedrockのテキスト生成機能をデモするために、Boto3 クライアントを使用して Amazon Bedrock API と通信する方法を確認します。さまざまな設定と、シンプルな入力で望ましい出力が得られるかを確認します。

#### パターン
Amazon Bedrock API に、タスク、指示、モデルへの入力から成る単純な入力を提供し、追加の例を提供することなく出力を生成させます。ここでの目的は、強力な LLM が与えられたタスクを理解し、説得力のある出力を生成できることをデモすることです。

![bedrock](./images/bedrock-code-gen.png)

#### ユースケース
Amazon Bedrock のモデルの生成機能をデモするために、メール生成のユースケースを取り上げましょう。

#### ペルソナ
あなたは AnyCompany のカスタマーサービスマネージャーのボブです。あなたの顧客の中には、カスタマーサポートエンジニアによって提供されたサービスについて否定的なフィードバックを提供している人がいます。今、あなたはそれらの顧客に謝罪して信頼を回復したいと考えています。わかりやすく、かつメールを送った顧客にパーソナライズされたメールを大量作成するのに、LLM の助けが必要です。

#### 実装
このユースケースを満たすために、このノートブックでは、顧客の以前のメールに基づいて、感謝のメールを生成する方法を示します。 Boto3 クライアントを使用した Amazon Bedrock API で、Amazon Titan Text Express モデルを使用します。

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww

# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## テキストの生成

上記のユースケースの説明に沿って、Amazon Bedrock サービスにメールを生成させるための入力を準備しましょう。このプロンプトは、[Human:/Assistant: formatting for Claude](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting) でフォーマットする必要があることに注意してください。

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


まず、Amazon Titan Text Express のモデルを使用しましょう。Amazon Titan Text Express は約 8k トークンのコンテキストウィンドウをサポートし、以下のパラメータを受け入れます:

- `inputText`:  LLM へのプロンプト
- `textGenerationConfig`: 出力を生成する際にモデルが考慮するパラメータ

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

Amazon Bedrock API の `invoke_model` API では以下のオプションが設定可能です:

- `modelId`: Amazon Bedrock で利用可能な各種基盤モデルのモデル ARN
- `accept`: 入力リクエストのタイプ
- `contentType`: 出力のコンテンツタイプ
- `body`: プロンプトと設定から成るJSON文字列

利用可能なテキスト生成モデルは [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) を確認してください。

#### Amazon Titan Text Express モデルの実行

最初に、どのようにモデルがどのようなアウトプットを生成するのかを先ほど作成したプロンプトで確認します。

##### アウトプットの生成

In [ ]:
modelId = 'amazon.titan-text-express-v1' # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# 応答部分は最初の改行文字'\n'の後から始まります
# 以下では最初の改行文字の後の応答を出力します

email = outputText[outputText.index('\n')+1:]
print_ww(email)


##### ストリーミングアウトプットの生成
上部の結果は、Amazon Titan モデルがインプットリクエストを理解し、異なる情報伝達の手法を理解する能力によって作成したサンプルのメールです。  
今回のリクエストは、同期的な API に対して行われ、モデルによって完全な出力が生成されるのを待ちます。

Bedrock はチャンク形式で作成されたモデルの出力をストリーミングすることもできます。  
以下は、ストリーミングオプションでのモデルの実行例です。  
`invoke_model_with_response_stream` API の応答 `ResponseStream` を読み込みます。

In [ ]:
output = []
try:
    
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

上記は、モデルの出力をすばやく取得し、読み込みながらサービスが完了させるのに役立ちます。  
これは、モデルにより長いテキストの生成を要求するユースケースで役立ちます。  
生成されたすべてのチャンクを組み合わせることで完全な出力が必要なケースにも対応できます。

In [ ]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

## まとめ
ここでは、Amazon Bedrock API を基本的な boto3 SDK で呼び出す方法を検証し、この API を使用し、顧客の否定的なフィードバックに対応してEメールを生成するユースケースに対応できることが確認できました。

### 重要なポイント
- このノートブックを使って、Amazon Bedrockで利用できるAnthropic ClaudeやAI21 Labs Jurassicなど、さまざまなモデルを試してみましょう。
- プロンプトを自分の具体的なユースケースに変更し、さまざまなモデルの出力を評価してみましょう。
- トークンの長さを変更して、サービスのレイテンシと反応性を理解してみましょう。
- より良い出力を得るために、さまざまなプロンプトエンジニアリングの原則を適用してみましょう。

## Thank You